In [23]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer   # 결측치값 대체.
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline

In [24]:
merged_df = pd.read_csv("../data/merged_dataset_ver.1.csv")
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32593 entries, 0 to 32592
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   code_module           32593 non-null  object 
 1   code_presentation     32593 non-null  object 
 2   id_student            32593 non-null  int64  
 3   gender                32593 non-null  object 
 4   region                32593 non-null  object 
 5   highest_education     32593 non-null  object 
 6   imd_band              31482 non-null  object 
 7   age_band              32593 non-null  object 
 8   num_of_prev_attempts  32593 non-null  int64  
 9   studied_credits       32593 non-null  int64  
 10  disability            32593 non-null  object 
 11  final_result          32593 non-null  object 
 12  date_registration     32548 non-null  float64
 13  date_unregistration   10072 non-null  float64
 14  sum_click             32593 non-null  float64
 15  avg_score          

In [25]:
#################
# 이상치 처리 
#################
# 조건: final_result가 Fail이고, date_unregistration이 null이 아님
condition1 = (merged_df["final_result"] == "Fail") & (merged_df["date_unregistration"].notnull())

# 해당 조건을 만족하는 행 제거
merged_df = merged_df[~condition1]

# 조건 정의: date_registration이 NaN이고 final_result가 Withdrawn인 경우
condition = (merged_df["date_registration"].isna()) & (merged_df["final_result"] == "Withdrawn")

# 해당 조건에 맞는 행 제거
merged_df = merged_df[~condition]
merged_df

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,date_registration,date_unregistration,sum_click,avg_score,banked_ratio,is_dropout
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,-159.0,NaN,934.0,82.000000,0.0,0
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,-53.0,NaN,1435.0,66.400000,0.0,0
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn,-92.0,12.0,281.0,0.000000,0.0,1
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,-52.0,NaN,2158.0,76.000000,0.0,0
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,-176.0,NaN,1034.0,54.400000,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32588,GGG,2014J,2640965,F,Wales,Lower Than A Level,10-20,0-35,0,30,N,Fail,-4.0,NaN,41.0,0.000000,0.0,0
32589,GGG,2014J,2645731,F,East Anglian Region,Lower Than A Level,40-50%,35-55,0,30,N,Distinction,-23.0,NaN,893.0,88.111111,0.0,0
32590,GGG,2014J,2648187,F,South Region,A Level or Equivalent,20-30%,0-35,0,30,Y,Pass,-129.0,NaN,312.0,76.666667,0.0,0
32591,GGG,2014J,2679821,F,South East Region,Lower Than A Level,90-100%,35-55,0,30,N,Withdrawn,-49.0,101.0,275.0,91.500000,0.0,1


In [ ]:
###############################결측치 처리####################################
# # #결측치 처리 컬럼: imd_band(범주형 -> 최빈값으로 처리), date_registration(수치형 -> mean으로 처리)
# # # date_unregisitration(9999값으로 채우기)

# # # 1. 결측치 직접 처리: date_unregistration → 9999
merged_df["date_unregistration"] = merged_df["date_unregistration"].fillna(9999)
merged_df

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,date_registration,date_unregistration,sum_click,avg_score,banked_ratio,is_dropout
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,-159.0,9999.0,934.0,82.000000,0.0,0
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,-53.0,9999.0,1435.0,66.400000,0.0,0
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn,-92.0,12.0,281.0,0.000000,0.0,1
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,-52.0,9999.0,2158.0,76.000000,0.0,0
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,-176.0,9999.0,1034.0,54.400000,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32588,GGG,2014J,2640965,F,Wales,Lower Than A Level,10-20,0-35,0,30,N,Fail,-4.0,9999.0,41.0,0.000000,0.0,0
32589,GGG,2014J,2645731,F,East Anglian Region,Lower Than A Level,40-50%,35-55,0,30,N,Distinction,-23.0,9999.0,893.0,88.111111,0.0,0
32590,GGG,2014J,2648187,F,South Region,A Level or Equivalent,20-30%,0-35,0,30,Y,Pass,-129.0,9999.0,312.0,76.666667,0.0,0
32591,GGG,2014J,2679821,F,South East Region,Lower Than A Level,90-100%,35-55,0,30,N,Withdrawn,-49.0,101.0,275.0,91.500000,0.0,1


In [ ]:
############################## 2. 인코딩 직접 처리######################################

# # 1) highest_education 학력별로 부여
education_order = {
    "No Formal Quals": 0,
    "Lower Than A Level": 1,
    "A Level Or Equivalent": 2,
    "He Qualification": 3,
    "Post Graduate Qualification": 4
}

# # 문자열 정리 후 replace 사용
merged_df["highest_education"] = merged_df["highest_education"].str.strip().str.title()
merged_df["highest_education"] = merged_df["highest_education"].replace(education_order)

C:\Users\Playdata\AppData\Local\Temp\ipykernel_20040\1193128172.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df["highest_education"] = merged_df["highest_education"].replace(education_order)


In [28]:
# # 2) age_band 중간 값으로 변경
age_map = {
    "0-35": 30,     # 또는 17.5, 20도 가능 (의도에 따라)
    "35-55": 45,
    "55<=": 60      # 55세 이상이므로 60 또는 65로 추정
}

merged_df["age_band"] = merged_df["age_band"].str.strip().str.title()
merged_df["age_band"] = merged_df["age_band"].replace(age_map)

C:\Users\Playdata\AppData\Local\Temp\ipykernel_20040\422327503.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df["age_band"] = merged_df["age_band"].replace(age_map)


In [29]:
# # # # # 3) 취약계층 서열화 
imd_order = {
    "0-10%": 1,
    "10-20": 2,
    "20-30%": 3,
    "30-40%": 4,
    "40-50%": 5,
    "50-60%": 6,
    "60-70%": 7,
    "70-80%": 8,
    "80-90%": 9,
    "90-100%": 10
}

merged_df["imd_band"] = merged_df["imd_band"].str.strip().str.title()
merged_df["imd_band"] = merged_df["imd_band"].replace(imd_order)

C:\Users\Playdata\AppData\Local\Temp\ipykernel_20040\638276554.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df["imd_band"] = merged_df["imd_band"].replace(imd_order)


In [30]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32545 entries, 0 to 32592
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   code_module           32545 non-null  object 
 1   code_presentation     32545 non-null  object 
 2   id_student            32545 non-null  int64  
 3   gender                32545 non-null  object 
 4   region                32545 non-null  object 
 5   highest_education     32545 non-null  int64  
 6   imd_band              31434 non-null  float64
 7   age_band              32545 non-null  int64  
 8   num_of_prev_attempts  32545 non-null  int64  
 9   studied_credits       32545 non-null  int64  
 10  disability            32545 non-null  object 
 11  final_result          32545 non-null  object 
 12  date_registration     32539 non-null  float64
 13  date_unregistration   32545 non-null  float64
 14  sum_click             32545 non-null  float64
 15  avg_score             32

In [31]:
# # # # # 1. 원래 컬럼 순서 저장
original_columns = merged_df.columns.tolist()

# 2. ColumnTransformer 설정  - nan값 대체
na_transformer = ColumnTransformer([
    ("category_imputer", SimpleImputer(strategy="most_frequent"), ['imd_band']),
    ("number_imputer", SimpleImputer(strategy="mean"), ['date_registration'])
], remainder="passthrough")

# 3. 처리 대상 및 passthrough 대상 정리
processed_columns = ['imd_band', 'date_registration']
passthrough_columns = [col for col in merged_df.columns if col not in processed_columns]

# 4. fit_transform 적용
na_values_array = na_transformer.fit_transform(merged_df)

# 5. DataFrame으로 변환 + 컬럼 순서 복원
merged_df = pd.DataFrame(na_values_array, columns=processed_columns + passthrough_columns)
merged_df = merged_df[original_columns]  # 순서 복원!
merged_df

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,date_registration,date_unregistration,sum_click,avg_score,banked_ratio,is_dropout
0,AAA,2013J,11391,M,East Anglian Region,3,10.0,60,0,240,N,Pass,-159.0,9999.0,934.0,82.0,0.0,0
1,AAA,2013J,28400,F,Scotland,3,3.0,45,0,60,N,Pass,-53.0,9999.0,1435.0,66.4,0.0,0
2,AAA,2013J,30268,F,North Western Region,2,4.0,45,0,60,Y,Withdrawn,-92.0,12.0,281.0,0.0,0.0,1
3,AAA,2013J,31604,F,South East Region,2,6.0,45,0,60,N,Pass,-52.0,9999.0,2158.0,76.0,0.0,0
4,AAA,2013J,32885,F,West Midlands Region,1,6.0,30,0,60,N,Pass,-176.0,9999.0,1034.0,54.4,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32540,GGG,2014J,2640965,F,Wales,1,2.0,30,0,30,N,Fail,-4.0,9999.0,41.0,0.0,0.0,0
32541,GGG,2014J,2645731,F,East Anglian Region,1,5.0,45,0,30,N,Distinction,-23.0,9999.0,893.0,88.111111,0.0,0
32542,GGG,2014J,2648187,F,South Region,2,3.0,30,0,30,Y,Pass,-129.0,9999.0,312.0,76.666667,0.0,0
32543,GGG,2014J,2679821,F,South East Region,1,10.0,45,0,30,N,Withdrawn,-49.0,101.0,275.0,91.5,0.0,1


In [32]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32545 entries, 0 to 32544
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   code_module           32545 non-null  object
 1   code_presentation     32545 non-null  object
 2   id_student            32545 non-null  object
 3   gender                32545 non-null  object
 4   region                32545 non-null  object
 5   highest_education     32545 non-null  object
 6   imd_band              32545 non-null  object
 7   age_band              32545 non-null  object
 8   num_of_prev_attempts  32545 non-null  object
 9   studied_credits       32545 non-null  object
 10  disability            32545 non-null  object
 11  final_result          32545 non-null  object
 12  date_registration     32545 non-null  object
 13  date_unregistration   32545 non-null  object
 14  sum_click             32545 non-null  object
 15  avg_score             32545 non-null

In [ ]:
#######################target 값 생성################################
#target ver2. (withdraw/fail) 
merged_df["target"] = merged_df["final_result"].map({
    "Pass": 0,
    "Distinction": 0,
    "Fail": 1,
    "Withdrawn": 1
})

# 변환되지 않은 값 확인
unmapped = merged_df[merged_df["target"].isna()]
if not unmapped.empty:
    print("⚠️ 변환되지 않은 값이 있습니다:")
    print(unmapped["final_result"].unique())


merged_df_wd_fail= merged_df.drop(columns=["final_result"])

In [ ]:
################################banked_ratio 필요없어서 삭제############################
target_wd_fail = pd.read_csv("../data/target_wd_fail.csv")
target_wd_fail= target_wd_fail.drop(columns=["banked_ratio"])
target_wd_fail.to_csv("../data/final_dataset.csv", index=False, encoding="utf-8-sig")